In [9]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

def fetch_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def preprocess_data(data):
    data["Close_Ratio_10"] = data["Close"] / data["Close"].rolling(10).mean()
    data["Trend_10"] = data["Close"].pct_change().rolling(10).sum()
    data["Tomorrow"] = data["Close"].shift(-1)
    data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
    data.dropna(inplace=True)
    return data

def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
    model.fit(X_train, y_train)
    return model

def predict(model, data, predictors):
    data["Predictions"] = model.predict(data[predictors])
    return data

def backtest(strategy_model, train_data, test_data, initial_capital=10000):
    capital = initial_capital
    positions = {}
    returns_percentage = []
    final_value = capital
    buy_dates = []
    trade_data = []

    # Training Phase
    strategy_model = train_model(train_data[predictors], train_data["Target"])

    # Out-of-Sample Testing
    test_data = predict(strategy_model, test_data, predictors)

    positions[symbol] = 0

    for i in range(len(test_data) - 1):
        if test_data["Predictions"].iloc[i] == 1:  # Buy signal
            shares_to_buy = int(round(capital / test_data["Close"].iloc[i])) - 1
            if shares_to_buy > 0:
                initial_value = capital
                capital -= shares_to_buy * test_data["Close"].iloc[i]
                positions[symbol] += shares_to_buy
                buy_dates.append((test_data.index[i], test_data["Close"].iloc[i], initial_value))
        elif test_data["Predictions"].iloc[i] == 0:  # Sell signal
            if buy_dates:
                capital += positions[symbol] * test_data["Close"].iloc[i]
                profit_loss = positions[symbol] * (test_data["Close"].iloc[i] - buy_dates[-1][1])
                trade_data.append([symbol, buy_dates[-1][0], buy_dates[-1][1], positions[symbol], test_data.index[i],
                                   test_data["Close"].iloc[i], profit_loss, buy_dates[-1][2]])
                positions[symbol] -= positions[symbol]
                buy_dates.pop()

        returns_percentage.append((capital / initial_capital - 1) * 100)

    final_value = capital + positions[symbol] * test_data["Close"].iloc[-1]

    return returns_percentage, final_value, trade_data

def fetch_nifty50_stocks(start_date, end_date):
    nifty50_symbols = [
    "BHARTIARTL.NS", "RELIANCE.NS", "TECHM.NS", "COALINDIA.NS", "INDUSINDBK.NS", "BAJAJ-AUTO.NS", "CIPLA.NS",
    "TCS.NS", "BAJAJFINSV.NS", "NTPC.NS", "TATASTEEL.NS", "ULTRACEMCO.NS", "KOTAKBANK.NS", "LT.NS", "MARUTI.NS",
    "NESTLEIND.NS", "HINDALCO.NS", "HDFCLIFE.NS", "WIPRO.NS", "TATACONSUM.NS", "BAJFINANCE.NS", "TITAN.NS",
    "BRITANNIA.NS", "ONGC.NS", "HEROMOTOCO.NS", "APOLLOHOSP.NS", "ITC.NS", "ADANIENT.NS", "LTIM.NS"
]
    nifty50_data = {}
    for symbol in nifty50_symbols:
        nifty50_data[symbol] = fetch_stock_data(symbol, start_date, end_date)
    return nifty50_data

# Example usage:
start_date = "2010-01-01"
end_date = "2023-12-11"

# Fetch NIFTY50 stock data
nifty50_data = fetch_nifty50_stocks(start_date, end_date)

# Define predictors
predictors = ["Close_Ratio_10", "Trend_10"]

# Initialize dictionaries to store precision scores, cumulative returns, final values, and trade data
precision_scores = {}
cumulative_returns = {}
final_values = {}
trade_data_dict = {}
stock_values = {}
performance= {}
i=0
# Train models and backtest for each stock
for symbol, stock_data in nifty50_data.items():
    processed_data = preprocess_data(stock_data)

    if not all(col in processed_data.columns for col in predictors + ["Target"]):
        print(f"Skipping {symbol} due to missing columns.")
        continue

    # Split data into training and testing sets
    train_data, test_data = train_test_split(processed_data, test_size=0.2, shuffle=False)

    # Backtest for the current stock
    returns_percentage, final_value, trade_data = backtest(None, train_data, test_data, initial_capital=10000)
    cumulative_returns[symbol] = returns_percentage
    final_values[symbol] = final_value
    trade_data_dict[symbol] = trade_data

    # Store initial and final values for each stock
    stock_values[symbol] = {"Initial Value": 10000, "Final Value": final_value}

    # Print trade details and final values for the current stock
    print(f"\nTrade Details for {symbol}:")
    trade_data_df = pd.DataFrame(trade_data,
                                 columns=["Symbol", "Buy Date", "Buy Price", "Shares Bought", "Sell Date", "Sell Price",
                                          "Profit/Loss", "Initial Value"])
    print(trade_data_df)
    print(f"Initial Capital: 10000.00")
    print(f"Final Capital: {final_value:.2f}")
    print(f"Percentage of Returns: {((final_value / 10000) - 1) * 100:.2f}%")
    if (((final_value / 10000) - 1) * 100) > 30:
        performance[i]={symbol}
        i=i+1
        
        

# Calculate and display overall initial and final values
all_initial_values = sum(stock["Initial Value"] for stock in stock_values.values())
all_final_values = sum(stock["Final Value"] for stock in stock_values.values())
overall_returns_percentage = ((all_final_values / all_initial_values) - 1) * 100

print("\nOverall Initial and Final Values:")
print(f"All Initial Values: {all_initial_values:.2f}")
print(f"All Final Values: {all_final_values:.2f}")
print(f"Overall Percentage of Returns: {overall_returns_percentage:.2f}%")
print("performance stocks:")
print(performance)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******